In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch


from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader
from tf_lab.nips.helper import center_pclouds_in_unit_sphere

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
n_pc_samples = 2048
do_training = True
first_time_running = True
load_model = False
load_epoch = None
seed = 42

max_training_epochs = 1000
loss = 'emd'

class_name = raw_input('Give me the class type: ').lower()
bottleneck = int(raw_input('Give me the bottle-neck size: '))

syn_id = shape_net_category_to_synth_id()[class_name]

experiment_name =  class_name + '_' + 'conv_arch_bneck_' + str(bottleneck) + '_' + str(n_pc_samples) +  'pts_' + loss

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/3d_neighbs_as_context/nn_models/ae_farming')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

Give me the class type: table
Give me the bottle-neck size: 128


'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/nn_models/ae_farming/table_conv_arch_bneck_128_2048pts_emd'

In [6]:
# Load Raw Point-Clouds of class
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
pclouds_path = osp.join(pclouds_path, syn_id)
file_names = pio.load_filenames_of_input_data(pclouds_path, '.ply')
pclouds, model_ids, syn_ids = pio.load_crude_point_clouds(file_names=file_names, n_threads=50, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(pclouds), )

train_data = PointCloudDataSet(pclouds)

8509 files containing complete point clouds were found.


In [ ]:
if n_pc_samples == 2048:
    encoder_args = {'n_filters': [128, 128, 256, bottleneck],
                    'filter_sizes' :[40, 20, 10, 10],
                    'strides': [1, 2, 2, 1]}
else:
    assert(False)
    
if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    
    
else:
    n_input = [n_pc_samples, 3]

    decoder_args = {'layer_sizes': [1024, 2048, np.prod(n_input)]}

    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = False, 
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args,        
               )

    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))    
    
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:

    if load_epoch is None: # load last saved.
        saved_epochs = read_saved_epochs(conf.train_dir)
        load_epoch = saved_epochs[-1]
        
    ae.restore_model(conf.train_dir, load_epoch, verbose=True)

In [ ]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.5058', 'loss=', '224.686668306')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/nn_models/ae_farming/table_conv_arch_bneck_128_2048pts_emd/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '0.4940', 'loss=', '155.719744469')
('Epoch:', '0003', 'training time (minutes)=', '0.4973', 'loss=', '138.770716364')
('Epoch:', '0004', 'training time (minutes)=', '0.5027', 'loss=', '131.692962287')
('Epoch:', '0005', 'training time (minutes)=', '0.5002', 'loss=', '126.029739245')
('Epoch:', '0006', 'training time (minutes)=', '0.5000', 'loss=', '123.284099713')
('Epoch:', '0007', 'training time (minutes)=', '0.5027', 'loss=', '117.822660154')
('Epoch:', '0008', 'training time (minutes)=', '0.5076', 'loss=', '115.878090802')
('Epoch:', '0009', 'training time (minutes)=', '0.5069', 'loss=', '113.639061243')
('Epoch:', '0010', 'training time (

In [ ]:
# Plot Some feed-reconstruction pairs.
feed = train_data.next_batch(1)[0]
rec = ae.reconstruct(feed)[0]
Point_Cloud(feed[0]).plot();
Point_Cloud(rec[0]).plot();